In [1]:
import pandas as pd

In [3]:
gambling_keywords_df = pd.read_csv("SlotSchool3NF_Tables/Gambling_Keywords.csv")
counties_df = pd.read_csv("SlotSchool3NF_Tables/Counties.csv")
gamblingplace_keywords_df = pd.read_csv("SlotSchool3NF_Tables/GamblingPlace_Keywords.csv")
gambling_places_df = pd.read_csv("SlotSchool3NF_Tables/Gambling_Places.csv")
addresses_df = pd.read_csv("SlotSchool3NF_Tables/SchoolAddresses.csv")
schools_df = pd.read_csv("SlotSchool3NF_Tables/Schools.csv")
school_place_proximity_df = pd.read_csv("SlotSchool3NF_Tables/School_Place_Proximity.csv")

In [ ]:
# rename school name coordinates to easily distinguish between the original name/address coordinates orginially in the schools csv
# schools_df.rename(columns={"SchoolLatitude": "SchoolNameLatitude"}, inplace=True)
# schools_df.rename(columns={"SchoolLongitude": "SchoolNameLongitude"}, inplace=True)

In [3]:
# remove trusted_location from GamblingPlaces
gambling_places_df = gambling_places_df.drop("TrustedLocation", axis=1)

In [4]:
gambling_places_df.to_csv("SlotSchool3NF_Tables/Gambling_Places.csv", index=False)

In [10]:
# dropped columns to satisfy BCNF
schools_df = schools_df.drop(columns=["SchoolNameLatitude", "SchoolNameLongitude", "CoordDistance"])

In [11]:
schools_df.to_csv("SlotSchool3NF_Tables/Schools.csv", index=False)

In [4]:
# Define suspected primary keys for each table and check if they are unique for BCNF
key_checks = {
    "gambling_keywords_df": gambling_keywords_df["KeywordId"].is_unique,
    "counties_df": counties_df["CountyId"].is_unique,
    "gamblingplace_keywords_df": (gamblingplace_keywords_df[["GamblingPlaceId", "KeywordId"]]
                                   .duplicated().sum() == 0),
    "gambling_places_df": gambling_places_df["GamblingPlaceId"].is_unique,
    "addresses_df": addresses_df["SchoolAddressId"].is_unique,
    "schools_df": schools_df["SchoolId"].is_unique,
    "school_place_proximity_df": (school_place_proximity_df[["SchoolId", "GamblingPlaceId"]]
                                   .duplicated().sum() == 0)
}

key_checks

{'gambling_keywords_df': True,
 'counties_df': True,
 'gamblingplace_keywords_df': np.True_,
 'gambling_places_df': True,
 'addresses_df': True,
 'schools_df': False,
 'school_place_proximity_df': np.True_}

In [5]:
# Load the CSV files for each table
# Adjust the file paths if needed
df_sp = pd.read_csv("SlotSchool3NF_Tables/School_Place_Proximity.csv")
df_schools = pd.read_csv("SlotSchool3NF_Tables/Schools.csv")
df_gambling = pd.read_csv("SlotSchool3NF_Tables/Gambling_Places.csv")

# --- Check 1: Ensure every SchoolId in school_place_proximity exists in schools ---
missing_school_ids = [~df_sp['SchoolId'].isin(df_schools['SchoolId'])]
if missing_school_ids.empty:
    print("All SchoolId values in school_place_proximity exist in the schools table.")
else:
    print("The following rows in school_place_proximity reference SchoolId values not found in the schools table:")
    print(missing_school_ids)

# --- Check 2: Ensure every GamblingPlaceId in school_place_proximity exists in gambling_places ---
missing_gambling_ids = df_sp[~df_sp['GamblingPlaceId'].isin(df_gambling['GamblingPlaceId'])]
if missing_gambling_ids.empty:
    print("All GamblingPlaceId values in school_place_proximity exist in the gambling_places table.")
else:
    print("The following rows in school_place_proximity reference GamblingPlaceId values not found in the gambling_places table:")
    print(missing_gambling_ids)

# --- Check 3 (Optional): Look for duplicate composite keys (SchoolId, GamblingPlaceId) ---
duplicate_rows = df_sp[df_sp.duplicated(subset=['SchoolId', 'GamblingPlaceId'], keep=False)]
if duplicate_rows.empty:
    print("There are no duplicate composite keys (SchoolId, GamblingPlaceId) in the school_place_proximity table.")
else:
    print("The following duplicate rows were found (based on SchoolId and GamblingPlaceId):")
    print(duplicate_rows)

AttributeError: 'list' object has no attribute 'empty'